<a href="https://colab.research.google.com/github/shmilyface/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 3

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame. ✓
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI). ✓
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na ✓
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary ✓

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [178]:
# 1a - Load data

import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [179]:
# 1b - validate # of observations

df.shape

(690, 16)

In [180]:
df.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12     object
13     object
14      int64
15     object
dtype: object

In [181]:
# 1c - clean & sanitize

df.replace('?', float('nan'), inplace=True)
df.dropna(inplace=True);
df.shape

(653, 16)

In [182]:
# 1d - just semantics

def colCast(col, to):
  df[col - 1] = df[col - 1].astype(to)

def numCats(col, vals):
  df[col - 1].replace(vals, range(len(vals)), inplace=True)

colCast(2, float)
colCast(11, float)
colCast(14, float)
colCast(15, float)

numCats(1, ['a', 'b'])
numCats(4, ['u', 'y', 'l', 't'])
numCats(5, ['g', 'p', 'gg'])
numCats(6, ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff'])
numCats(7, ['v', 'h', 'bb', 'j', 'n', 'z', 'dd', 'ff', 'o'])
numCats(9, ['t', 'f'])
numCats(10, ['t', 'f'])
numCats(12, ['t', 'f'])
numCats(13, ['g', 'p', 's'])
numCats(16, ['+', '-'])

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,30.83,0.000,0,0,9,0,1.25,0,0,1.0,1,0,202.0,0.0,0
1,0,58.67,4.460,0,0,8,1,3.04,0,0,6.0,1,0,43.0,560.0,0
2,0,24.50,0.500,0,0,8,1,1.50,0,1,0.0,1,0,280.0,824.0,0
3,1,27.83,1.540,0,0,9,0,3.75,0,0,5.0,0,0,100.0,3.0,0
4,1,20.17,5.625,0,0,9,0,1.71,0,1,0.0,1,2,120.0,0.0,0


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [183]:
# Step 1: Split into + and -
dfPlus = df[df[15] == 1]
dfPlus.drop(15, axis=1, inplace=True)

dfMinus = df[df[15] == 0]
dfMinus.drop(15, axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [207]:
# Step 2a: Visualize categorical data
import matplotlib.pyplot as plt

catsPlus = dfPlus.drop([1, 2, 7, 10, 13, 14], axis=1)
catsMinus = dfMinus.drop([1, 2, 7, 10, 13, 14], axis=1)
contPlus = dfPlus.drop([0, 3, 4, 5, 6, 8, 9, 11, 12], axis=1)
contMinus = dfMinus.drop([0, 3, 4, 5, 6, 8, 9, 11, 12], axis=1)

catsPlus[0]
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(catsPlus[0].value_counts().values())
fig.show()


#for c in catsPlus.columns:
  
#  plt.add_subplot()
#  plt.bar(c + 100, range(len(catsPlus[c].value_counts().keys())), [v for k, v in catsPlus[c].value_counts()])
#plt.bar(range(len(catsPlus.columns)), [catsPlus[c].sum() / len(catsPlus[c]) for c in catsPlus.columns])

70     1
72     0
73     1
74     1
75     1
76     1
77     0
78     1
79     1
80     1
81     0
82     1
84     1
85     1
87     1
88     1
89     0
90     1
91     1
93     1
94     1
95     0
96     1
98     0
99     0
100    1
101    1
102    1
103    1
104    1
      ..
659    0
660    1
661    1
662    0
663    1
664    1
665    1
666    0
667    0
668    1
669    1
670    1
671    1
672    0
674    0
675    0
676    0
677    1
678    0
679    0
680    1
681    1
682    1
683    1
684    1
685    1
686    0
687    0
688    1
689    1
Name: 0, Length: 357, dtype: int64

In [227]:
# not very different - A1
# very different - A3

dfPlus[2].sum() / dfPlus.shape[0]

3.882324929971989

In [228]:
dfMinus[2].sum() / dfMinus.shape[0]

5.9719425675675675

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

*Your words here!*